#### Física de la Radioterapia. Master de Física Biomédica. Universidad Complutense de Madrid

# Registro de imágenes en radioterapia
## Transformación de estructuras. Acumulación de dosis
------

*Este cuaderno está basado en los cuadernos de ejemplo del módulo* [`platipy`](https://pyplati.github.io/platipy/getting_started.html)

Los datos corresponden a un paciente real tratado en el Hospital de Fuenlabrada.

Han sido anonimizados de acuerdo a procedimientos validados.

El paciente se somete a un tratamiento de próstata que tiene que incluir además de la glándula prostática, las vesículas seminales y los ganglios pélvicos. El tratamiento se prescribe en 28 sesiones. La distribución de dosis se integra en todos los volúmenes a lo largo del tratamiento completo. Las dosis de prescripción son 70  Gy a la próstata, 2.5  Gy por fracción, 56  Gy a las vesículas seminales, 2  Gy por fracción, y 50.4  Gy a las cadenas ganglionares pélvicas, 1.8  Gy por fracción. Los protocolos determinan que el paciente se trate con la vejiga llena y el recto vacío.

Los criterios para la aceptacicón del plan son los siguientes

| Estructura	| Medida |Unidad	| Objetivo |
|----|-----|----|----|
| PTV prostata | V66.50 Gy | [% of volume] |  > 95 |
| PTV prostata | V74.90 Gy | [% of volume] | < 5 |
| PTV vvss | V53.20 Gy | [% of volume] |  > 95 |
| PTV vvss | V59.95 Gy | [% of volume] | < 5 |
| PTV gg | V47.88 Gy | [% of volume] |  > 95 |
| PTV gg | V53.93 Gy | [% of volume] | < 5 |
| Recto	| Dosis media | Gy	|	< 44 |
| Recto | V36.4 Gy | [% of volume] | < 60 |
| Recto | V45.4 Gy | [% of volume] | < 50 |
| Recto | V54.5 Gy | [% of volume] | < 35 |
| Recto | V59 Gy | [% of volume] | < 25 |
| Recto | V63.6 Gy | [% of volume] | < 20 |
| Recto | V68.2 Gy | [% of volume] | < 15 |
| Vejiga | V59 Gy | [% of volume] | < 50 |
| Vejiga | V63.60 Gy | [% of volume] | < 35 |
| Vejiga | V68.20 Gy | [% of volume] | < 25 |
| Vejiga | V72.70 Gy | [% of volume] | < 15 |
| Cabeza femoral D | Dosis máxima | Gy | < 54 |
| Cabeza femoral D | V45 Gy | [% of volume] | < 10 |
| Cabeza femoral I | Dosis máxima | Gy | < 54 |
| Cabeza femoral I | V45 Gy | [% of volume] | < 10 |
| Intestino | V40 Gy | [cm³] | < 124 |
| Intestino | V45 Gy | [cm³] | < 71 |
| Intestino | V60 Gy | [cm³] | < 0.50 |
| Intestino | Dosis máxima | Gy | < 55 |



El paciente presenta un llenado adecuado de la vejiga a la largo del tratamiento pero los oncólogos observan una *variación en la posición* de la vejiga. Después de las primeras 18 sesiones de tratamiento deciden realizar un segundo escáner y una nueva planificación.

**Objetivos**:

- Mostrar de forma práctica los conceptos de registro de imagen, tanto rígido como deformable.

- Emplear las tranformacioneos obtendidas para mostrar su aplicación en la delimitación de estructuras y en la acumulación de dosis.

Los módulos necesarios que no están por defecto en **Colab** son:
- **platipy**: utilidades de registro y análisis de planes
- **pyplastimatch**: *navaja suiza* de estudios DICOM en radioterapia

> `platipy` tiene como dependencia **SimpleITK**, una biblioteca de software de código abierto para el análisis de imágenes, que proporciona una interfaz sencilla para las poderosas herramientas de procesamiento de imágenes de ITK, ideal para la manipulación y el procesamiento de imágenes médicas.

>Las utilidades **plastimatch** solo serán necesarias si se desea exportar los cálculos a openTPS para una facilitar su visualización en 3D y navegar por los estudios con mayor versatilidad. En el [Apéndice](##Apéndice) se explica cómo utilizarlo.


In [ ]:
!uv pip install platipy pyplastimatch
import os
import SimpleITK as sitk

Descargamos de la nube los datos que vamos a emplear.

Los datos se almacenan en la carpeta `/content/data` dentro de nuestra máquina **Colab**. Son por tanto volátiles, se borrarán cuando el entorno de ejecución se desconecte. Téngase en cuenta también que la carpeta `/content` es a la que apunta por defecto el explorador de archivos en **Colab**, la carpeta `data` que aparecerá en nuestro explorador será con la que trabajaremos.

`data` contiene los dos conjuntos completos de datos de radioterapia de las dos simulaciones: CT, estructuras, plan y dosis.

In [ ]:
%%bash

# Crear el directorio de datos
rm -rf /content/data
mkdir -p /content/data

# Ejecutar código Python para descargar los datos del Drive
python << EOF > /dev/null 2>&1
# Descargar los datos compartidos de Google Drive
import gdown
url = 'https://drive.google.com/file/d/1YMc4ix4WyGEMVNef-3r8uDWBwkElEpe-/view?usp=drive_link'
output = '/content/data/ResimulacionProstata.tgz'
gdown.download(url, output, quiet=True, fuzzy=True)
EOF

# Preparación de los datos para realziar el cuaderno de Registro de imágenes

# Extraer archivo de datos
cd /content/data
tar -xvzf ResimulacionProstata.tgz >/dev/null

# Eliminar el archivo comprimido
rm -f /content/data/ResimulacionProstata.tgz

#### Carga de imágenes CT

Cargamos las imágenes CT de la **Planificación** (que usaremos como referencia o imagen fija) y de la  **Replanificación** (que usaremos como imagen móvil). Utilizaremos `SimpleITK.ImageSeriesReader` para leer las series DICOM desde los directorios correspondientes.

> El CT de Planificación lo denotamos `fixed` y todos sus datos asociados llevarán en prefijo `fixed_`. El CT de Replanificación lo denotamos `moving` y todos sus datos asociados llevarán el prefijo `moving_`.

In [ ]:
# Objeto lector
reader = sitk.ImageSeriesReader()

# Leer las imágenes del CT de planificación inicial
dicom_names = reader.GetGDCMSeriesFileNames('/content/data/PatientID/Planificacion/CT')
reader.SetFileNames(dicom_names)
fixed = reader.Execute()

# Leer las imágenes del CT de la replanificación
dicom_names = reader.GetGDCMSeriesFileNames('/content/data/PatientID/Replanificacion/CT')
reader.SetFileNames(dicom_names)
moving = reader.Execute()

#### Interpretación espacial de las imágenes
Para representar dos imágenes coherentemente necesitamos que ambas imágenes estén referidas a un mismo marco de referencia y compartan el mismo muestreo espacial. En Geometría diríamos que ambas imágenes tienen el mismo sistema de coordenadas y la misma escala.

En Imagen Médica el marco de referencia puede venir referido al equipo de adquisión de imagen, o si se prefiere al espacio de la sala donde está instalado el equipo. Estas coordenadas se denominan coordenadas universales. La posición del paciente en el equipo cambia el valor absoluto de las coordenadas universales. El marco de referencia también puede estar referido a la propia anatomía del paciente y en este caso se las refiere como coordenadas del paciente.

### Tarea
----
Modificar la plantilla de código
```
from platipy.imaging import ImageVisualiser

image_visualiser = ImageVisualiser(image1)

image_visualiser.add_comparison_overlay(image)

fig = image_visualiser.show()
```
para ver si es posible representar conjuntamente las imágenes `fixed` y `moving` que hemos creado.

Si no es posible por un problema de coherencia entre ellas resolverlo utilizando la plantilla de código

```
image2_resampled = sitk.Resample(image2, image1)
```

para obtener un remuestreo de la imagen `moving` en relación a `fixed`

In [ ]:
# Añadir las celdas necesarias para realizar la tarea.

### Registro Lineal (Rígido)

El registro lineal, a menudo conocido como registro rígido o afín, busca encontrar una transformación que alinee globalmente una imagen móvil (`moving`) a una imagen fija (`fixed`) mediante operaciones lineales como traslación, rotación, escala y cizallamiento. Este tipo de registro es útil para corregir desplazamientos y rotaciones generales entre imágenes.

En `platipy`, la función `linear_registration` se utiliza para realizar este tipo de registro. Requiere la imagen fija y la imagen móvil como entradas y devuelve la imagen móvil transformada (alineada) y el objeto de transformación que describe el movimiento encontrado. Con la opción `verbose=True` se puede tener una idea del progreso del algoritmo de optimización durante el registro.

In [ ]:
from platipy.imaging.registration.linear import linear_registration

### Tarea
-----
Modificar la plantilla de código
```
aligned, align_transform = linear_registration(fixed_image=image1, moving_image=image2, verbose=True)
```
- Para realizar un registro rígido de la imagen `moving` respecto a la imagen `fixed`.
- Representar conjuntamente las imágenes `fixed` y `aligned`
> El resultado `aligned` ya está remuestreado respecto a `fixed`.

In [ ]:
# Añadir las celdas necesarias para realizar la tarea

### Cuestión
----
1. Realizar una evaluación cualitativa sobre si el registro ha producido una mejora en el alineamiento de las imágenes y de las límitaciones de esta modalidad de registro para conseguir la mejor superposición posible.

### Respuesta
----
**Tu respuesta aquí**

## Registro deformable de imagen (DRI)
El registro deformable de imágenes es una técnica avanzada utilizada para alinear dos imágenes de manera no rígida, permitiendo que las estructuras se deformen localmente para lograr una correspondencia precisa entre ellas. A diferencia del registro rígido, puede corregir distorsiones complejas y variaciones anatómicas.

Uno de los algoritmos más conocidos y utilizados para el registro deformable es el algoritmo **Demons**. El algoritmo Demons (o **Symmetric Forces Demons**) funciona tratando cada píxel o vóxel de la imagen en movimiento como un "demonio" que empuja o tira de su correspondiente en la imagen fija, basándose en la diferencia de intensidad entre ellos. Este proceso iterativo genera un campo de desplazamiento que deforma la imagen móvil hasta que se alinea con la imagen fija.

El algoritmo está implementado en el módulo `platipy`



In [ ]:
from platipy.imaging.registration.deformable import fast_symmetric_forces_demons_registration

### Tarea
----
Modificar la siguiente plantilla de código
```
warp_transform = fast_symmetric_forces_demons_registration(image1, image2, verbose=True)
```

para realizar un registro deformable entre las imágenes `fixed`  y `moving`

Visualizar conjuntamente las dos imágenes `fixed` y `adapted`

> La imagen `adapted` (imagen `moving` adaptada por el algoritmo Demons para coincidir con la imagen `fixed`) la devuelve la función `fast_symmetric_forces_demons_registration` como primer elemento del objeto devuelto `warp_transform[0]`

In [ ]:
# Añadir las celdas necesarias para realizar la tarea

### Cuestión
----
2. Realizar una evaluación cualitativa sobre si el registro deformable ha producido una mejora en el alineamiento de las imágenes respecto al registro rígido y sobre su capacidad para superar las límitaciones indentificadas previamente en el registro rígido.

### Respuesta
----
**Tu respuesta aquí**

## Aplicación del registro a la transformación de estructuras
Una vez establecido un registro que conecte dos imágenes, es posible aplicarlo para transformar cualquier información ligada a la geometría de una de ellas a la otra.

Así ocurre por ejemplo con las estructuras contorneadas para delimitar volúmenes y órganos de riesgo.


Importamos las funciones del módulo `platipy` que nos harán falta para trabajar con las estructuras

In [ ]:
from platipy.dicom.io.rtstruct_to_nifti import convert_rtstruct
from platipy.imaging.label.utils import get_com

Convertimos las estructuras de formato **DICOM** a formato `nifti` para el estudio de planificación (Plan) como para el de replanificación (Replan)

In [ ]:
output_dir = '/content/data/PatientID/nifti/Plan/ss'
!rm -rf $output_dir
!mkdir -p $output_dir
convert_rtstruct(dcm_img='/content/data/PatientID/Planificacion/CT',
                 dcm_rt_file='/content/data/PatientID/Planificacion/Estructuras/RS.1.2.246.352.205.5317915818984065047.16825537653637026489.dcm',
                 output_dir=output_dir,
                 prefix="")
volumefiles = [f for f in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, f))]
fixed_ss = {}
for volumefile in volumefiles:
    _name = volumefile.split(".nii.gz")[0]
    fixed_ss[_name] = sitk.ReadImage(os.path.join(output_dir, volumefile))

In [ ]:
output_dir = '/content/data/PatientID/nifti/Replan/ss'
!rm -rf $output_dir
!mkdir -p $output_dir
convert_rtstruct(dcm_img='/content/data/PatientID/Replanificacion/CT',
                 dcm_rt_file='/content/data/PatientID/Replanificacion/Estructuras/RS.1.2.246.352.205.5437916376192068699.2843408499515398533.dcm',
                 output_dir=output_dir,
                 prefix="")
volumefiles = [f for f in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, f))]
moving_ss = {}
for volumefile in volumefiles:
    _name = volumefile.split(".nii.gz")[0]
    moving_ss[_name] = sitk.ReadImage(os.path.join(output_dir, volumefile))

En lo que sigue, por razones de simplicidad, no consideramos todas las estructuras contorneadas. Creamos diccionarios con las estructuras relevantes: los CTV de próstata y vesículas, el recto y la vejiga

In [ ]:
ss_to_show = ['CTV_prostata', 'CTV_vvss', 'Vejiga', 'Recto']
fixed_sss = {key: fixed_ss[key] for key in ss_to_show if key in fixed_ss}
moving_sss = {key: moving_ss[key] for key in ss_to_show if key in moving_ss}

Representamos la imagen de planificación con sus estructuras relevantes superpuestas

In [ ]:
image_visualiser = ImageVisualiser(fixed, cut=get_com(fixed_ss["PTV_prostata"]))
image_visualiser.add_contour(fixed_sss)
fig = image_visualiser.show()

Podemos representar también las imágenes de planficación superponiendo en bruto las estructuras contorneadas en la imagen de replanificación

In [ ]:
image_visualiser = ImageVisualiser(fixed, cut=get_com(fixed_ss["PTV_prostata"]))
image_visualiser.add_contour(moving_sss)
fig = image_visualiser.show()

### Transformaciones por registro rígido
Las diferencias anatómicas que pueden presentar estas estructuras son difíciles de evaluar porque sabemos que hay también una diferencia de marco de referencia entre los dos conjuntos de datos.

Para poder hacer una primera evaluación tenemos que introducir una corrección de alineación entre ambos marcos. En geometría hablaríamos de un cambio de base por traslación y giro. Puede aparecer también una cambio de métrica en el caso en el que el campo de visión se haya cambiado en el escáner.

El módulo `platipy` implementa la función `apply_linear_transform` para aplicar la transformación lineal obtenida de un registro rígido a cualquier otra imagen que tenga el mismo marco de referencia utilizado en el registro.

In [ ]:
from platipy.imaging.registration.utils import apply_linear_transform

### Tarea
----
- Modificar la plantilla de código

```
aligned_ss = apply_linear_transform(input_image=moving_ss, reference_image=image, transform=transform, is_structure=True)
```

para modificar las estructuras referidas al marco `moving` y convertirlas al marco `fixed`.
> `transform` ha sido previamtente obtenida al realizar el registro rígido entre ambas imágenes.

- Realizar una representación conjunta de la imagen `fixed` con las estructuras contorneadas en `moving` pero alineadas con `fixed`

In [ ]:
# Añadir las celdas necesarias para realizar la tarea

### Cuestión
----
3. Realizar una evaluación cualitativa tanto del grado de alineamiento como del grado de coincidencia anatómica entre las estructuras una vez desplazadas y la imagen.

### Respuesta
----
**Tu respuesta aquí**


### Trasformaciones por registro deformable
Las limitaciones introducidas por las variaciones anatómicas son superables utilizando la transformación obtenida mediante el registro deformable.

El módulo `platipy` implementa la función `apply_deformable_transform` para apicar el campo de deformación que conecta dos conjuntos de imagen y transformar cualquier información referida a una de ellos al otro.

In [ ]:
from platipy.imaging.registration.utils import apply_deformable_transform

### Tarea
----
- Modificar la plantilla de código
```
adapted_ss = apply_transform(input_image=aligned_ss, reference_image=image1, transform=transform)
```

para adaptar las estructuras contorneadas en `moving` a la anatomía en la imagen `fixed`.

> Téngase en cuenta que como queremos obtener las estructuras en el marco de referencia `fixed` tendremos que utilizar las estructuras contorneadas en `moving` pero referidas (alineadas) a `fixed`

- Realizar una representación de la imagen `fixed` con las estructuras `moving` adaptadas

In [ ]:
# Añadir las celdas necesarias para realizar la tarea

### Cuestión
----
4. Realizar una evaluación cualitativa sobre si aplicar la transformación del registro deformable ha reducido la influencia de las variaciones anatómicas en la superposición de las estructuras con la imagen.

### Respuesta
----
**Tu respuesta aquí**

## Evaluación cuantitativa del grado de acuerdo entre estructuras
Idealmente, dos conjuntos de estructuras contorneadas sobre la misma imagen deberían coincidir.

En la práctica hay incertidumbres que hacen que los conjuntos no se superpongan completamente. Estas incertidumbres provienen del sistema de delimitación, sea humano, inteligencia artificial o basado en la combinación de atlas (bases de datos de pacientes) más registro deformable.

Se han desarrollado índices que miden el grado de coincidencia entre conjuntos de estructuras.

El módulo `platipy` implementa funciones para realizar la determinación de varios de estos índices.

In [ ]:
from platipy.imaging.visualisation.comparison import contour_comparison
from platipy.imaging.utils.crop import label_to_roi

La siguiente celda realiza una comparación cuantitativa entre el conjunto de estructuras delimitado sobre la imagen `fixed` y el conjunto delimitado sobre la imagen `moving` y alineaddo con `fixed`

In [ ]:
(sag_size, cor_size, ax_size), (sag_0, cor_0, ax_0) = label_to_roi(
            fixed_ss["Recto"] | fixed_ss["Vejiga"], expansion_mm=40
        )
limits = [ax_0,
          ax_0 + ax_size,
          cor_0,
          cor_0 + cor_size,
          sag_0,
          sag_0 + sag_size,
]

for ss in ss_to_show:
  fixed_ss[ss].SetOrigin(aligned_ss[ss].GetOrigin())
  fixed_ss[ss].SetSpacing(aligned_ss[ss].GetSpacing())
  fixed_ss[ss].SetDirection(aligned_ss[ss].GetDirection())

fig, aligned_cc_df = contour_comparison(
    img = fixed,
    contour_dict_a = fixed_sss,
    contour_dict_b = aligned_ss,
    contour_label_a = "Plan inicial",
    contour_label_b = "Alineado",
    structure_for_com = "Recto",
    title='Comparación de estructuras',
    subtitle='Registro rígido',
    subsubtitle='Transformación lineal',
    img_vis_kw={"limits": limits},
)

### Tarea
----
Duplicar a continuación la celda anterior y modificar el código para hacer una comparación de las estructuras contorneadas sobre la imagen `moving` adaptadas a la anatomía de `fixed` con las estructuras delimitadas directamente sobre la imagen `fixed`.


In [ ]:
# Modificar esta celda para realizar la tarea

### Cuestión
----
5. Realizar una comparción cuantitativa de la ganancia en el grado de acuerdo entre los conjuntos de estructuras al pasar del registro rígido al registro deformable.
> Consultar la página de ejemplo [`contour_comparison`](https://pyplati.github.io/platipy/_examples/contour_comparison.html) del módulo `platipy` y la documentación referida en ella para obtener una indicación del significado de los índices medidos por la función `contour_comparison`

### Respuesta
----
**Tu respuesta aquí**

## Deformación de la dosis

En tanto la dosis va ligada al tejido que la ha recibido, para realizar una suma coherente de la dosis acumulada en diferentes sesiones en las que se producen variaciones anatómicas, es necesario deformar la dosis en la misma medida en la que se deforman los tejidos.

En las siguientes celdas realizamos la lectura, escalado y representación de las dosis en la situación `fixed`
> Realizamos el mismo procedimiento que ya se mostró en la entrega sobre **Evaluación de dosis y Radiobiología**

In [ ]:
import pydicom as dcm
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
dose_file_path = '/content/data/PatientID/Planificacion/Dosis/RD.1.2.246.352.71.7.964456977994.687683.20251104082750.dcm'
fixed_dose = sitk.ReadImage(dose_file_path)
fixed_dose = sitk.Resample(fixed_dose, fixed)
fixed_dose_dataset = dcm.dcmread(dose_file_path)
fixed_dose = sitk.Cast(fixed_dose, sitk.sitkFloat32) * fixed_dose_dataset.DoseGridScaling

In [ ]:
image_visualiser = ImageVisualiser(fixed, cut=get_com(fixed_ss["PTV_prostata"]))
image_visualiser.add_contour(fixed_sss)
image_visualiser.add_scalar_overlay(fixed_dose, discrete_levels=20, colormap=matplotlib.colormaps.get_cmap("jet"), name="Dose (Gy)")
fig = image_visualiser.show()

En las siguientes celdas leemos y escalamos la dosis calculada en el momento de la replanificación, situación `moving`. El factor adicional de escala de 28/10 se introduce porque la replanificación se hizo en la sesión 19, cuando quedaban 10 de las 28 sesiones proyectadas. La dosis se calculó para esas 10 sesiones restantes. Para compararla con la dosis inicial introducimos esta corrección.

Mediante `apply_linear_transform` cambiamos el marco de referencia para que la dosis esté referida a la situación `fixed`
> Hay que tener en cuenta que a parte de la corrección por desplazamiento, la dosis se calculó para la situación anatómica de la situación `moving`

In [ ]:
dose_file_path = '/content/data/PatientID/Replanificacion/Dosis/RD.1.2.246.352.71.7.964456977994.675637.20250822191728.dcm'
moving_dose = sitk.ReadImage(dose_file_path)
moving_dose_dataset = dcm.dcmread(dose_file_path)
moving_dose = sitk.Cast(moving_dose, sitk.sitkFloat32) * moving_dose_dataset.DoseGridScaling * 28 / 10
aligned_dose = apply_linear_transform(input_image=moving_dose, reference_image=fixed, transform=align_transform, is_structure=False)

In [ ]:
image_visualiser = ImageVisualiser(fixed, cut=get_com(moving_ss["PTV_prostata"]))
image_visualiser.add_contour(fixed_sss)
image_visualiser.add_scalar_overlay(aligned_dose, discrete_levels=20, colormap=matplotlib.colormaps.get_cmap("jet"), name="Dose (Gy)")
fig = image_visualiser.show()

### Cuestión
----
6. Relizar un evaluación **desde el punto de vista físico** sobre la adecuación de la distribución de dosis *alineada* a la situación `fixed`, tanto para el paciente en su conjunto como para la región en la que el PTV de próstata se solapa con la vejiga.

### Respuesta
----
**Tu respuesta aquí**

Para una adecuación más realista de la distribución física de la dosis tenemos que aplicar una deformación no rígida de forma similar a como hemos hecho con las estructuras

### Tarea
----
- Escribir el código necesario para deformar `aligned_dose` mediante el campo de deformación que obtuvimos al registrar la imagen `moving` a la imagen `fixed`

- Hacer una representación similar a la anterior en la que se muestre la dosis adaptada sobre la imagen `fixed`

In [ ]:
# Añadir las celdas necesarias para realizar la tarea

### Cuestión
----
7. Reevaluar **desde el punto de vista físico** la adecuación de la distribución de dosis *adaptada* a la situación `fixed`, tanto para el paciente en su conjunto como para la región en la que el PTV de próstata se solapa con la vejiga.

### Respuesta
----
**Tu respuesta aquí**

## Acumulación de dosis
Una estimación de la acumulación de dosis a medida que se dan las sucesivas sesiones de tratamiento se puede obtener mediante la suma de las correspondientes distibuciones de dosis recibidas por el paciente referidas a un mismo marco de referencia.

La estimación será tanto más realista cuanto mejor sea nuestra capacidad para seguir y corregir las perturbaciones, tanto globales como locales, que se introducen en el tratamiento, si bien hay que tenemos que ser conscientes de que estos cálculos siempre implican un cierto grado de incertidumbre.

Desde un punto de vista histórico, cuando las herramientas de registro deformable no estaban disponibles en la práctica clínica común, al producirse una replanificación, en el mejor de los casos se podía estimar una dosis acumulada por registro rígido, que tenía que ser interptretada individualmente, al ser válida únicamente desde el punto de vista físico en la región en la que la coincidencia del registro fuera adecuada.

Las herramientas de registro deformable han permitido realizar una mejor acumulación de dosis.

En lo que sigue vamos a suponer que el tratamiento se ha realizado en las dos situaciones ya referidas, `fixed`y `moving`, y que la mitad de la dosis se ha dado en una situación y el resto de la dosis en la otra.

### Tarea
- Añadir código para realizar estimaciones de la dosis acumulada a partir de las dosis alineada y adaptada según las plantillas
```
estimated_dose = fixed_dose / 2 + aligned_dose / 2
acumulated_dose = fixed_dose / 2 + adapted_dose / 2

```
- Realizar representaciones de las dosis estimadas y acumuladas sobre la imagen `fixed`

In [ ]:
# Añadir las celdas necesarias para realizar la tarea

## Evaluación de la dosis acumulada mediante DVHs
En las siguientes celdas se calculan los histogramas para las dosis `estimated_dose`, `acumulated_dose` y la dosis originalmente planificada

In [ ]:
from platipy.imaging.dose.dvh import calculate_dvh_for_labels, calculate_d_x, calculate_v_x

In [ ]:
fig, ax = plt.subplots()

dvh = calculate_dvh_for_labels(estimated_dose, fixed_sss)
dvh = dvh.set_index("label")
dvh = dvh.rename_axis("Dose")
dvh = dvh.iloc[:,3:].transpose()
dvh = dvh * 100
dvh.plot(ax=ax, kind="line", style='--', colormap=matplotlib.colormaps.get_cmap("rainbow"), legend=False)

dvh = calculate_dvh_for_labels(acumulated_dose, fixed_sss)
dvh = dvh.set_index("label")
dvh = dvh.rename_axis("Dose")
dvh = dvh.iloc[:,3:].transpose()
dvh = dvh * 100
dvh.plot(ax=ax, kind="line", style='-.', colormap=matplotlib.colormaps.get_cmap("rainbow"), legend=False)

dvh = calculate_dvh_for_labels(fixed_dose, fixed_sss)
dvh = dvh.set_index("label")
dvh = dvh.rename_axis("Dose")
dvh = dvh.iloc[:,3:].transpose()
dvh = dvh * 100
dvh.plot(ax=ax, kind="line", colormap=matplotlib.colormaps.get_cmap("rainbow"), legend=False)

# Add labels and show plot
plt.legend(loc='best')
plt.xlabel("Dose (Gy)")
plt.ylabel("Volume [%]")
plt.title("Dose Volume Histogram (DVH)")
plt.show()

### Cuestión
----
8. Comentar cualitativamente las diferencias entre los DVHs.

### Respuesta
----
**Tu respuesta aquí**

## Apéndice

### `plastimatch`

`plastimatch` es una herramienta de código abierto para el procesamiento de imágenes médicas, especialmente útil para el registro y la segmentación de imágenes en radioterapia, incluyendo la conversión *robusta* entre diferentes formatos de imagen médica como DICOM y NIfTI.

El sitio [`plastimatch`](https://plastimatch.org/) proporciona binarios para varias [**plataformas**](https://plastimatch.org/getting_started.html#getting-started). Se puede instalar en Ubuntu Linux (variante Debian) mediante:
```
apt install plastimatch
```

Sin embargo, no siempre está disponible el binario adecuado para la versión de la máquina **Colab**.

> **Colab** es un contenedor que corre un sistema Ubuntu

El módulo **pyplastimatch** es un envoltorio (*wrapper*) para utilizar **plastimatch** desde **python**.
Afortunadamente los desarrolladores de **pyplastimatch** incorporan una utilidad `install_precompiled_binaries` que nos resuelve la dificultad de instalar el binario adecuado para nuestra plataforma.

En la siguiente celda instalamos el módulo y el binario de plastimatch

  


In [ ]:
!uv pip install pyplastimatch
from pyplastimatch.utils.install import install_precompiled_binaries
install_precompiled_binaries()



### Conversión `nifti` -> `DICOM` mediante `plastimatch`

Los objetos que hemos creado mediante el módulo `platipy` por transformaciones derivadas de registros son de tipo `SimpleITK image`.

La siguiente celda muestra a modo de ejemplo el tipo de alguno de los objetos con los que hemos estado trabajando

In [ ]:
type(fixed), type(moving), type(aligned), type(adapted), type(aligned_dose), type(adapted_dose), type(adapted_ss['Recto'])

El módulo `SimpleITK` permite exportar todos estos objetos en formato **nifti** entre otros muchos.
> También es posible exportar en formato **DICOM**, pero considerando todos los objetos que componen un conjunto completo de datos de radioterapia (CT, estructuras, dosis) es más compacto y robusto hacerlo mediante `plastimatch`

La siguiente celda muestra el código necesario para realizar la exportación de los objetos del conjunto completo de datos a formato **nifti**

In [ ]:
from SimpleITK import WriteImage
!mkdir -p /content/data/PatientID/nifti/adapted/ss
WriteImage(adapted, '/content/data/PatientID/nifti/adapted/adapted.nii.gz')
WriteImage(adapted_dose, '/content/data/PatientID/nifti/adapted/adapted_dose.nii.gz')
for ss in ss_to_show:
  WriteImage(adapted_ss[ss], f'/content/data/PatientID/nifti/adapted/ss/{ss}.nii.gz')

La conversión mediante `plastimatch` de **nifti** a **DICOM** se realiza mediante el siguiente código
> Ver la documentación del comando **plastimatch**  [**convert**](https://plastimatch.org/plastimatch.html) y el cuaderno de ejemplo de uso de [**pyplastimatch**](https://colab.research.google.com/github/AIM-Harvard/pyplastimatch/blob/main/notebooks/pyplastimatch_MWE.ipynb) para entender estas opciones y ver la posibilidad de muchas otras.

In [ ]:
rt_study_path = '/content/data/PatientID/Replanificacion/aligned'
convert_args = {'input' : '/content/data/PatientID/nifti/adapted/adapted.nii.gz',
                'input-dose-img' : '/content/data/PatientID/nifti/adapted/adapted_dose.nii.gz',
                'input-prefix' : '/content/data/PatientID/nifti/adapted/ss',
                'output-dicom' : rt_study_path}

from pyplastimatch import convert
convert(verbose=True, **convert_args)